In [ ]:
from pickle import dump
import mysql.connector
import pandas as pd
#import dask.dataframe as dd
from sklearn.model_selection import train_test_split
import missingno as msno
from time import time
import numpy as np
import matplotlib.pyplot as plt
from collections import defaultdict

## Connect to database

In [ ]:
cnx = mysql.connector.connect(user='****', password='****', host='127.0.0.1', database='urls')
cursor = cnx.cursor()

In [ ]:
cursor.execute("SELECT table_name FROM information_schema.tables WHERE table_schema = 'urls'")
tables_list = [tables[0] for tables in cursor.fetchall()]

In [ ]:
tokens_tables = list(filter(lambda x: "token" in x, tables_list))
data_tables = list(filter(lambda x: "table" in x, tables_list))

In [ ]:
tokens_tables

In [ ]:
tables = []
for i in data_tables:
    query = "SELECT * FROM %s"%(i)
    tables.append(pd.read_sql_query(query, con=cnx))

In [ ]:
tokens = []
for i in tokens_tables:
    query = "SELECT * FROM %s"%(i)
    tokens.append(pd.read_sql_query(query, con=cnx))

In [ ]:
cnx.commit()
cursor.close()
cnx.close()

In [ ]:
#Concat all tokens db in one
num_urls = sum([token.shape[0] for token in tokens])
db_tokens = pd.concat(tokens, sort=False, ignore_index=True)
db_tokens.drop_duplicates(subset=["url"], inplace=True)
print("Size of complete tokens database: ", db_tokens.shape)
print("Num of URLS should be: ", num_urls)
assert(num_urls == db_tokens.shape[0])


In [ ]:
"""condition  = db_tokens['url'].duplicated()
print(db_tokens.index[condition])
print(db_tokens.iloc[2085]["url"])"""


In [ ]:
#Concat all table db in one
num_urls = sum([table.shape[0] for table in tables])
db_tables = pd.concat(tables, sort=False, ignore_index=True)
db_tables.drop_duplicates(subset=["url"], inplace=True)
print("Size of complete tables database: ", db_tables.shape)
print("Num of URLS should be: ", num_urls)
assert(num_urls == db_tables.shape[0])

In [ ]:
#Delete tokens urls that don't appear on table
url_list = list(db_tables["url"])
condition  = ~db_tokens['url'].isin(url_list)
delete_list = db_tokens.index[condition]
db_tokens.drop(delete_list, inplace=True)
db_tokens.reset_index(drop=True, inplace=True)

#Delete table urls that don't appear on tokens
url_list = list(db_tokens["url"])
condition  = ~db_tables['url'].isin(url_list)
delete_list = db_tables.index[condition]
db_tables.drop(delete_list, inplace=True)
db_tables.reset_index(drop=True, inplace=True)

assert (db_tokens.shape[0]==db_tables.shape[0])

In [ ]:
#If some error in the tokens db correct it
"""from FeatureExtraction import *
from SQL_Manager import *


url_list = list(db_tokens["url"])
condition  = ~db_tables['url'].isin(url_list)
delete_list = db_tables.index[condition]
tokens_table = "Tokens7"
for ind in (delete_list):
    
    url = db_tables.iloc[ind]["url"]
    malicious = db_tables.iloc[ind]["malicious"].item()
    tokens_dict={}
    tokens_dict["url"] = url
    tokens_dict["malicious"] = malicious
    lexical_dict, t_dict = get_lexicalf(url)
    #t_dict.astype(int)
    tokens_dict.update(t_dict)
    print(type(db_tables.iloc[ind]["malicious"]))
    print(tokens_dict)

    tres = make_insertion(tokens_dict, tokens_table)
    if tres==1:
        tokens_table = tokens_table[:6]+str(int(tokens_table[6])+1)
        create_table(tokens_table)
        tres = make_insertion(tokens_dict, tokens_table)"""

In [ ]:
db_tables.info()

## Manage NULL values

In [ ]:
#msno.matrix(db_tables)

In [ ]:
#Drop columns that only have nulls in tokens
prev_col = db_tokens.shape[1]
db_tokens.dropna(how="all", axis=1, inplace=True)
print("Deleted columns: ", prev_col - db_tokens.shape[1])
# In tokens databases fill of 0
db_tokens.fillna(0, inplace=True)
print("Tokens databse filled with 0s")

In [ ]:
#Drop columns that only have nulls in tables
prev_col = db_tables.shape[1]
db_tables.dropna(how="all", axis=1, inplace=True)
print("Deleted columns: ", prev_col - db_tables.shape[1])

## Clean variables

In [ ]:
print(db_tables.dtypes.value_counts())

In [ ]:
#Solve dtypes errors
def replace_OS(x):
    if(type(x) == str): x = x.split("/")[0]
    return x
db_tables["h_contlen"] = db_tables["h_contlen"].apply(pd.to_numeric)
db_tables["h_lastmod"] = db_tables["h_lastmod"].apply(pd.to_datetime, errors="coerce", utc=True).dt.tz_localize(None)
db_tables["server_OS"] = db_tables["server_OS"].apply(lambda x: replace_OS(x))
print(db_tables.dtypes.value_counts())

In [ ]:
str_cols = []
date_cols = []
numeric_cols = {}
for col in db_tables.columns:
    if db_tables[col].dtype == "object":            #object, datetime64[ns], float64, int64
        str_cols.append(col)
    if db_tables[col].dtype == "datetime64[ns]":
        date_cols.append(col)
    if db_tables[col].dtype in ("float64", "int64"):
        numeric_cols[col]=0

In [ ]:
#Numeric columns are filled with 0s
print("Numeric null before: ", db_tables[numeric_cols.keys()].isna().sum().sum())
db_tables.fillna(value=numeric_cols, inplace=True)
print("Numeric null after: ", db_tables[numeric_cols.keys()].isna().sum().sum())

In [ ]:
freq_cols = []
for col in numeric_cols:
    if "freq_" in col:
        freq_cols.append(col)
    
for col in freq_cols:
    db_tables[col] = round(db_tables[col]/db_tables["len_url"],3)


In [ ]:
for col in (str_cols):
    if col in ['full_text', 'domain','path', 'complete_path', 'host_country', 'query', 'url', 'subdomain', 'final_url', 'netloc', 'scheme']:
        continue
    if col in list(filter(lambda x: "IP" in x, str_cols)):
        continue
    if col in list(filter(lambda x: "PTR" in x, str_cols)):
        continue
    print(col, db_tables[col].nunique())
print("Feature;")
#print(db_tables["server_OS"].value_counts())

In [ ]:
#String columns
print("String null before: ", db_tables[str_cols].isna().sum().sum())
print("Shape before: ", db_tables.shape)

# Columns to encode
encoding_cols = [c for c in str_cols if c not in list(filter(lambda x: "IP" in x, str_cols))]
encoding_cols = [c for c in encoding_cols if c not in list(filter(lambda x: "PTR" in x, str_cols))]
encoding_cols = [c for c in encoding_cols if c not in ['full_text', 'domain','path', 'complete_path', 'host_country', 'query','subdomain', 'final_url', 'netloc', 'scheme']]
prefixes = {}
for c in encoding_cols:
    if c=="url":
        continue
    if "PTR" in c:
        prefixes[c] = "PTR"
    else:
        prefixes[c] = c

# Columns to drop
delete_columns = [c for c in str_cols if c not in encoding_cols]
db_tables.drop(columns=delete_columns, inplace=True)

#One Hot encoding
encoding_cols = [c for c in encoding_cols if c != "url"]
db_tables = pd.get_dummies(db_tables, prefix=prefixes, columns=encoding_cols, drop_first=False)

print("Shape after: ", db_tables.shape)
print("String null after: ", db_tables.isna().sum().sum() - db_tables[date_cols].isna().sum().sum())

In [ ]:
#Date columns
print("Date null before: ", db_tables[date_cols].isna().sum().sum())
df = pd.DataFrame()
new_date_cols ={}
for col in date_cols:
    df[col+"_year"] = db_tables[col].dt.year
    new_date_cols[col+"_year"] = 3000
    df[col+"_month"] = db_tables[col].dt.month
    new_date_cols[col+"_month"] = 20
    df[col+"_week"]= db_tables[col].dt.isocalendar()["week"]
    new_date_cols[col+"_week"] = 60
    df[col+"_day"] = db_tables[col].dt.day
    new_date_cols[col+"_day"] = 40
    df[col+"_hour"] = db_tables[col].dt.hour
    new_date_cols[col+"_hour"] = 30
    df[col+"_minute"] = db_tables[col].dt.minute
    new_date_cols[col+"_minute"] = 70
    df[col+"_dayofweek"] = db_tables[col].dt.isocalendar()["day"]
    new_date_cols[col+"_dayofweek"] = 10

   
df.fillna(value = new_date_cols, inplace=True)
db_tables[list(new_date_cols.keys())]=df[list(new_date_cols.keys())]

print("Date null after: ", db_tables[new_date_cols].isna().sum().sum())

In [ ]:
db_tables.drop(columns=date_cols, inplace=True)
print("Total final null: ", db_tables.isna().sum().sum())

## Create training datasets

In [ ]:
cols_list = list(db_tables)
cols_list.remove("url")
cols_list.remove("malicious")
X_table = db_tables[cols_list]   
y_table = db_tables["malicious"]

In [ ]:
cols_list = list(db_tokens)
cols_list.remove("url")
cols_list.remove("malicious")
X_tokens = db_tokens[cols_list]    
y_tokens = db_tokens["malicious"]  

## ML classification algorithms

In [ ]:
#Import preprocessing tools
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

#Import classifiers
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import NeighborhoodComponentsAnalysis
from xgboost import XGBClassifier
from sklearn.calibration import CalibratedClassifierCV
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.svm import LinearSVC


#Import feature selection
#from sklearn.feature_selection import SequentialFeatureSelector
from mlxtend.feature_selection import SequentialFeatureSelector
from mlxtend.plotting import plot_sequential_feature_selection as plot_sfs
from sklearn.feature_selection import VarianceThreshold
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import SelectPercentile
from sklearn.decomposition import PCA
from sklearn.feature_selection import chi2, f_classif   #f_classif=ANOVA

#Import evaluation metrics
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import fbeta_score
from sklearn.metrics import recall_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_score
from sklearn.metrics import make_scorer
from sklearn.metrics import brier_score_loss
from sklearn.metrics import log_loss
from sklearn.model_selection import cross_validate
from sklearn.model_selection import GridSearchCV
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingGridSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.calibration import CalibrationDisplay

In [ ]:
#Previous dimensionality reduction on tokens
skx = SelectKBest(chi2, k=50)      
skx_tokens = skx.fit(X_tokens,y_tokens)     #reduce to just 700 more important bag of words tokens
token_cols = skx_tokens.get_feature_names_out()
token_cols = np.append(token_cols,"malicious")
token_cols = np.append(token_cols,"url")
db_tokens_reduce = db_tokens[token_cols]

#Create new db
join_db = db_tables.merge(db_tokens_reduce, how="outer") #outer
print("Columns reduction by CHI2: ", (db_tokens.shape[1]+db_tables.shape[1]-2)-(db_tokens_reduce.shape[1]+db_tables.shape[1]-2))
print("Columns addition: ", db_tokens_reduce.shape[1]+db_tables.shape[1]-2)
assert(db_tokens_reduce.shape[1]+db_tables.shape[1]-2 == join_db.shape[1] )
print("Total final null: ", join_db.isna().sum().sum())
print("Full Dataset Shape", join_db.shape)

cols_list = list(join_db)
cols_list.remove("url")
cols_list.remove("malicious")
X = join_db[cols_list]    
y = join_db["malicious"]  


#Filter best features
 
spa = SelectPercentile(f_classif, percentile=60)
X= spa.fit_transform(X,y)
final_features = spa.get_feature_names_out()
print("After ANOVA feature select", X.shape)
#Standard Scaler
scaler = StandardScaler()
scaler.fit(X,y)
X = scaler.transform(X)

t = time()
nca = NeighborhoodComponentsAnalysis(tol=0.001, max_iter=60, n_components=70, random_state=41)
X_NCA = nca.fit_transform(X,y)
print("After NCA space reduction", X_NCA.shape)
print("NCA time: ", time()-t)

In [ ]:
feat = spa.feature_names_in_
values = spa.pvalues_

enumerate_object = enumerate(values)
sorted_pairs = sorted(enumerate_object, key=lambda elem: elem[1])

sorted_indices = []
for index, element in sorted_pairs:
    sorted_indices.append(index)

feat_dict = defaultdict(list)
c=0
for i in (sorted_indices):
    feat_dict["Feature"].append(spa.feature_names_in_[i])
    feat_dict["pvalue"].append(spa.pvalues_[i])
    c+=1
    
    feat_db = pd.DataFrame(feat_dict)


    


feat_db.head(20)
#feat_db.to_excel('top_features_ANOVA.xlsx')

In [ ]:
def confusion_matrix_scorer(clf, X, y):
    y_pred = clf.predict(X)
    tn, fp, fn, tp = confusion_matrix(y, y_pred).ravel()
    return {'tn': tn, 'fp': fp,
            'fn': fn, 'tp': tp}


scoring = {"accuracy":make_scorer(accuracy_score),
            "fbeta":make_scorer(fbeta_score,beta=2), 
            "recall":make_scorer(recall_score) ,
            "precision":make_scorer(precision_score) ,
            "auroc":make_scorer(roc_auc_score, needs_proba=True)}

def metrics(ground_truth, prediction, predict_proba):
    conf_mat = confusion_matrix(ground_truth, prediction, labels=[1,0])
    acc = accuracy_score(ground_truth,prediction)
    recall = recall_score(ground_truth,prediction)
    precision= precision_score(ground_truth,prediction)
    fbeta=fbeta_score(ground_truth,prediction,beta=2)
    auroc=roc_auc_score(ground_truth, predict_proba[:, 1])
    brier = brier_score_loss(ground_truth,predict_proba[:, 1])
    logloss = log_loss(ground_truth,predict_proba)
    return [("Confusion Matrix: ", conf_mat),("Accuracy: ", acc), ("Recall: ", recall),("Precision: ", precision), ("F2-value: ", fbeta), 
    ("AUROC: ", auroc), ("Brier: ", brier), ("LogLoss: ", logloss)]


#### SVM

In [ ]:
rbfSVM_pipe = Pipeline([("svc",SVC())])

param_grid = {"svc__C": [10, 50, 100, 1000],
                "svc__kernel":["rbf"],
                "svc__probability":[True],
                "svc__gamma":["scale","auto", 1, 0.1, 0.01, 0.001, 0.0001]}
              

hgs = HalvingGridSearchCV(estimator=rbfSVM_pipe, param_grid=param_grid, scoring=scoring["accuracy"], factor=3, n_jobs=-1 , refit=False, cv=3, verbose=4, min_resources="exhaust")
hgs = hgs.fit(X, y)
rbfSVM_pipe.set_params(**hgs.best_params_)


rbf_svm_result = cross_validate(rbfSVM_pipe, X, y, cv=6, scoring=scoring, n_jobs=-1, verbose=4)
results = pd.DataFrame(rbf_svm_result)
results.loc['mean'] = results.mean()
results.loc['std'] = results.std()


print(rbfSVM_pipe)
results.head(8)


In [ ]:
rbfSVM_pipe = Pipeline([("svc",SVC())])

param_grid = {"svc__C": [10, 50, 100, 1000],
                "svc__kernel":["rbf"],
                "svc__probability":[True],
                "svc__gamma":["scale","auto", 1, 0.1, 0.01, 0.001, 0.0001]}
              

hgs = HalvingGridSearchCV(estimator=rbfSVM_pipe, param_grid=param_grid, scoring=scoring["accuracy"], factor=3, n_jobs=-1 , refit=False, cv=3, verbose=4, min_resources="exhaust")
hgs = hgs.fit(X_NCA, y)
rbfSVM_pipe.set_params(**hgs.best_params_)

rbf_svm_result = cross_validate(rbfSVM_pipe, X_NCA, y, cv=6, scoring=scoring, n_jobs=-1, verbose=4)
results = pd.DataFrame(rbf_svm_result)
results.loc['mean'] = results.mean()
results.loc['std'] = results.std()

print(rbfSVM_pipe)
results.head(8)


In [ ]:
lSVM_pipe = Pipeline([("svc",SVC())])
#lSVM_pipe = Pipeline([("svc",LinearSVC())])

param_grid = {"svc__C": [10, 50, 100, 1000],
                "svc__kernel":["linear"],
                "svc__probability":[True]}

"""param_grid = {"svc__C": [100, 1000],
                "svc__max_iter":range(1000,3000,100)}"""
               

hgs = HalvingGridSearchCV(estimator=lSVM_pipe, param_grid=param_grid, scoring=scoring["accuracy"], factor=3, n_jobs=-1 , refit=False, cv=3, verbose=4, min_resources=500)
hgs = hgs.fit(X_NCA, y)
lSVM_pipe.set_params(**hgs.best_params_)

"""scoring_linear = {"accuracy":make_scorer(accuracy_score),
           "fbeta":make_scorer(fbeta_score,beta=2), 
            "recall":make_scorer(recall_score) ,
            "precision":make_scorer(precision_score)}"""

print("Best params for linearSVM:", hgs.best_params_)
linear_svm_result = cross_validate(lSVM_pipe, X_NCA, y, cv=6, scoring=scoring, n_jobs=-1, verbose=4)
results = pd.DataFrame(linear_svm_result)
results.loc['mean'] = results.mean()
results.loc['std'] = results.std()

print(lSVM_pipe)
results.head(8)



#### Decision Trees

In [ ]:
dtpipe = Pipeline([("dt", DecisionTreeClassifier())])

param_grid = {"dt__criterion": ["gini"],
                "dt__max_depth": range(30,100,5),
                "dt__min_impurity_decrease":[0.1, 0.5 ,0.05],
                "dt__min_samples_split":range(2,30,2)}
               

hgs = HalvingGridSearchCV(estimator=dtpipe, param_grid=param_grid, scoring=scoring["fbeta"], factor=3, n_jobs=-1 , refit=False, cv=3, verbose=4, min_resources=300)
hgs = hgs.fit(X, y)
dtpipe.set_params(**hgs.best_params_)

print("Best params for rbfSVM:", hgs.best_params_)
dt_result = cross_validate(dtpipe, X, y, cv=6, scoring=scoring, n_jobs=-1, verbose=4)
results = pd.DataFrame(dt_result)
results.loc['mean'] = results.mean()
results.loc['std'] = results.std()

print(dtpipe)
results.head(8)


#### Nearest Neighbors

In [ ]:
knn_pipe = Pipeline([("knn",KNeighborsClassifier())])
param_grid = {"knn__n_neighbors": range(5,50,1),
                "knn__weights":["uniform"]}
               

hgs = HalvingGridSearchCV(estimator=knn_pipe, param_grid=param_grid, scoring=scoring["accuracy"], factor=3, n_jobs=-1 , refit=False, cv=3, verbose=4, min_resources=300)
hgs = hgs.fit(X, y)
knn_pipe.set_params(**hgs.best_params_)

print("Best params for rbfSVM:", hgs.best_params_)
knn_result = cross_validate(knn_pipe, X, y, cv=6, scoring=scoring, n_jobs=-1, verbose=4)
results = pd.DataFrame(knn_result)
results.loc['mean'] = results.mean()
results.loc['std'] = results.std()

print(knn_pipe)
results.head(8)


In [ ]:
#knn_pipe_nca = Pipeline([("scaler",scaler), ("nca", nca), ("knn",KNeighborsClassifier())])
knn_pipe_nca = Pipeline([("knn",KNeighborsClassifier())])

param_grid = {"knn__n_neighbors": range(5,50,1),
                "knn__weights":["uniform","distance"]}
               

hgs = HalvingGridSearchCV(estimator=knn_pipe_nca, param_grid=param_grid, scoring=scoring["fbeta"], factor=3, n_jobs=-1 , refit=False, cv=3, verbose=4, min_resources=300)
hgs = hgs.fit(X_NCA, y)
knn_pipe_nca.set_params(**hgs.best_params_)

print("Best params for rbfSVM:", hgs.best_params_)
knn_nca_result = cross_validate(knn_pipe_nca, X_NCA, y, cv=6, scoring=scoring, n_jobs=-1, verbose=4)
results = pd.DataFrame(knn_nca_result)
results.loc['mean'] = results.mean()
results.loc['std'] = results.std()

print(knn_pipe_nca)
results.head(8)


#### Ensamble Methods

In [ ]:
#Fit best estimators on all the data
clf_list = [("rbfSVM", rbfSVM_pipe),
            ("Decision Tree", dtpipe),
            ("kNN", knn_pipe_nca)]

In [ ]:
bagg_pipe = Pipeline([("bgg",BaggingClassifier(base_estimator=SVC(),n_jobs=-1, random_state=40))])

param_grid = {"bgg__n_estimators": range(10,30,5),
                "bgg__max_samples":[p/10 for p in range(2, 10,2)],
                "bgg__max_features":[p/10 for p in range(2, 10,2)]}
               

hgs = HalvingGridSearchCV(estimator=bagg_pipe, param_grid=param_grid, scoring=scoring["fbeta"], factor=3, n_jobs=-1 , refit=False, cv=3, verbose=4, min_resources=300)
hgs = hgs.fit(X_NCA, y)
bagg_pipe.set_params(**hgs.best_params_)

bagg_result = cross_validate(bagg_pipe, X_NCA, y, cv=6, scoring=scoring, n_jobs=-1, verbose=4)
results = pd.DataFrame(bagg_result)
results.loc['mean'] = results.mean()
results.loc['std'] = results.std()

print(bagg_pipe)
clf_list.append(("Bagging", bagg_pipe))
results.head(8)


In [ ]:
rfpipe = Pipeline([("rf",RandomForestClassifier(n_estimators= 100, max_features="sqrt", random_state=40))])

param_grid = {"rf__n_estimators": range(90,100,1),
                "rf__max_features": ['sqrt', 'log2',None]}
               

#hgs = HalvingGridSearchCV(estimator=rfpipe, param_grid=param_grid, scoring=scoring["accuracy"], factor=3, n_jobs=-1 , refit=False, cv=3, verbose=4, min_resources=200)
#hgs = hgs.fit(X, y)
#rfpipe.set_params(**hgs.best_params_)

rf_result = cross_validate(rfpipe, X, y, cv=6, scoring=scoring, n_jobs=-1, verbose=4)
results = pd.DataFrame(rf_result)
results.loc['mean'] = results.mean()
results.loc['std'] = results.std()

print(rfpipe)
clf_list.append(("Random Forest", rfpipe))
results.head(8)

In [ ]:
ada_pipe = Pipeline([("ada",AdaBoostClassifier(random_state=40))])

param_grid = {"ada__n_estimators": range(50,100,10),
            "ada__learning_rate": [0.1,0.5,1,3,5,10]}
               

hgs = HalvingGridSearchCV(estimator=ada_pipe, param_grid=param_grid, scoring=scoring["accuracy"], factor=3, n_jobs=-1 , refit=False, cv=3, verbose=4, min_resources=300)
hgs = hgs.fit(X, y)
ada_pipe.set_params(**hgs.best_params_)

ada_result = cross_validate(ada_pipe, X, y, cv=6, scoring=scoring, n_jobs=-1, verbose=4)
results = pd.DataFrame(ada_result)
results.loc['mean'] = results.mean()
results.loc['std'] = results.std()

print(ada_pipe)
clf_list.append(("AdaBoost", ada_pipe))
results.head(8)


In [ ]:
xgb_pipe = Pipeline([("xgb",XGBClassifier(random_state=40))])

param_grid = {"xgb__n_estimators": range(50,100,10),
            "xgb__learning_rate": [0.1,0.5,1,3,5,10]}
               

hgs = HalvingGridSearchCV(estimator=xgb_pipe, param_grid=param_grid, scoring=scoring["accuracy"], factor=3, n_jobs=-1 , refit=False, cv=3, verbose=4, min_resources=300)
hgs = hgs.fit(X, y)
xgb_pipe.set_params(**hgs.best_params_)

ada_result = cross_validate(xgb_pipe, X, y, cv=6, scoring=scoring, n_jobs=-1, verbose=4)
results = pd.DataFrame(ada_result)
results.loc['mean'] = results.mean()
results.loc['std'] = results.std()

print(xgb_pipe)
clf_list.append(("XGBoost", xgb_pipe))
results.head(8)


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_NCA, y, test_size=0.30, random_state=123)

In [ ]:
vc = VotingClassifier(estimators=clf_list, weights=[4,2,2,4,1,2,1], voting="soft")

vc = vc.fit(X_train, y_train)
print(metrics(y_test,vc.predict(X_test), vc.predict_proba(X_test)))


In [ ]:
vc = VotingClassifier(estimators=clf_list, weights=[4,1,1,4,3,1,1], voting="soft")
vc_result = cross_validate(vc, X_NCA, y, cv=6, scoring=scoring, verbose=4)
results = pd.DataFrame(vc_result)
results.loc['mean'] = results.mean()
results.loc['std'] = results.std()
results.head(8)

#### Probability calibration

In [ ]:
prob_clf_list = []
for (name,clf) in clf_list:
    #Platt
    platt = CalibratedClassifierCV(clf, method="sigmoid", cv=6, n_jobs=-1)
    prob_clf_list.append((name+"+ Platt",platt))
    iso = CalibratedClassifierCV(clf, method="isotonic", cv=6, n_jobs=-1)
    prob_clf_list.append((name+"+ Isotonic",iso))

In [ ]:

scores = defaultdict(list)
for i, (name, clf) in enumerate(prob_clf_list):
    clf.fit(X_train, y_train)
    
    prob_yhat = clf.predict_proba(X_test)
    yhat = clf.predict(X_test)
    scores["Classifier"].append(name)
    
    values = metrics(y_test, yhat, prob_yhat)

    for (metric,value) in values:
        scores[metric].append(value)

    score_df = pd.DataFrame(scores).set_index("Classifier")
    score_df.round(decimals=5)

score_df

In [ ]:
scoring = {"accuracy":make_scorer(accuracy_score),
            "fbeta":make_scorer(fbeta_score,beta=2), 
            "recall":make_scorer(recall_score) ,
            "precision":make_scorer(precision_score) ,
            "auroc":make_scorer(roc_auc_score, needs_proba=True),
            "brier":make_scorer(brier_score_loss, greater_is_better=False, needs_proba=True),
            "log_loss":make_scorer(log_loss, greater_is_better=False, needs_proba=True)}

scores = defaultdict(list)
for i, (name, clf) in enumerate(prob_clf_list):
    
    res = cross_validate(clf, X_NCA, y, cv=6, scoring=scoring, n_jobs=-1, verbose=4)
    results = pd.DataFrame(res)
    cols = list(results)
    
    scores["Classifier"].append(name)
    for (c,mean) in enumerate(results.mean()):
        scores[cols[c]].append(mean)

    score_df = pd.DataFrame(scores).set_index("Classifier")
    score_df.round(decimals=5)

score_df

## Results plot and graphical analysis

In [ ]:
#Plot calibration curves
alg_list = [prob_clf_list[a] for a in range(0,len(prob_clf_list),2)]
for j, (algorithm,x) in enumerate(alg_list):
    
    f=j*2
    s=f+2
    fig = plt.figure(figsize=(20, 20))
    colors = plt.cm.get_cmap("Dark2")
    ax = fig.add_subplot()

    calibration_displays = {}
    for i, (name, clf) in enumerate(prob_clf_list[f:s]):
        clf.fit(X_train, y_train)
        display = CalibrationDisplay.from_estimator(
            clf,
            X_test,
            y_test,
            n_bins=5,
            name=name,
            ax=ax,
            color=colors(i),
        )
        calibration_displays[name] = display

    plt.title("Calibration Curves (%s)"%(algorithm.split("+")[0]))
    plt.show()

In [ ]:
from sklearn.manifold import TSNE
import seaborn as sns
tsne_results = TSNE(n_components=2, learning_rate='auto', init='random').fit_transform(X)

x_tsne = tsne_results[:,0]
y_tsne = tsne_results[:,1]

sns.scatterplot(
    x=x_tsne, y=y_tsne,
    hue=y,
    palette=sns.color_palette("hls", 2),
    legend="full",
    alpha=0.3
)

In [ ]:
from sklearn.manifold import TSNE
import seaborn as sns
tsne_results = TSNE(n_components=2, learning_rate='auto', init='random').fit_transform(X_NCA)

x_tsne = tsne_results[:,0]
y_tsne = tsne_results[:,1]

sns.scatterplot(
    x=x_tsne, y=y_tsne,
    hue=y,
    palette=sns.color_palette("hls", 2),
    legend="full",
    alpha=0.3
)

## Save model

In [ ]:
#dump(final_features, open('model_columns', 'wb'))      #Save columns
#dump(clf, open('model.pkl', 'wb'))      #Save model